## **Before you start**

Make a copy of this Colab by clicking on File > Save a Copy in Drive

After making a copy, add your student id, last name, and first name to the title.

In [14]:
student_name = "Salvador Robles Herrera"
student_id = "80683116"

# Exercises Naive Bayes Classifier

## Naive Bayes - Example

Consider a dataset with 3 classes and 5 binary attributes. Let the arrays p_class and p_att_given_class be:

In [15]:
import numpy as np

p_class = np.ones(3)/3
# Probabilities that a given attribute of a class is 1
# Class 1 can be dogs, class 2 can be cat, class 3 can be like pandas
p_att_given_class = np.array([[0.72, 0.21, 0.89, 0.47, 0.64],[0.32, 0.82, 0.54, 0.82, 0.17],[0.76, 0.65, 0.74, 0.31, 0.75]])

print(p_class)
print()
print(p_att_given_class)


[0.33333333 0.33333333 0.33333333]

[[0.72 0.21 0.89 0.47 0.64]
 [0.32 0.82 0.54 0.82 0.17]
 [0.76 0.65 0.74 0.31 0.75]]


where p_class[i] represents the probability that an example belongs to class i and p_att_given_class[i,j] 
represents the probability that attribute j in an example of class i is equal to 1. 

What is the probability that for an example of class 2, attribute 0 is equal to 1?

In [16]:
print(p_att_given_class[2,0])

0.76


What is the probability that for an example of class 1, attribute 4 is equal to 0?

In [17]:
print(1-p_att_given_class[1,4])

0.83


How would the Naïve Bayes classifier classify example [1,1,1,0,0]? 

In [18]:
def classify(x,pc,pac):
  p = pc 
  #print('x',x)
  #print('p', p)
  #print('pac', pac)
  for i in range(x.shape[0]): # Number of attributes
    if x[i]==1:
      p = p* pac[:,i]
    else:
      p = p*(1-pac[:,i])

  #print('pi: ', p)
  #print('sum: ', np.sum(p))
  if np.sum(p) != 0:
    p = p/np.sum(p)
  #print('probs ', p)
  return np.argmax(p)


In [19]:
def classify_no_loops(x,pc,pac):
  p = x*pac + (1-x)*(1-pac)
  p = np.prod(p, axis=1)
  p = pc*p
  p = p/np.sum(p)
  print(p)
  return np.argmax(p)

In [20]:
test_example = np.array([1,1,1,0,0])
classify(test_example,p_class,p_att_given_class)

2

In [21]:
test_example = np.array([1,1,1,0,0])

z = classify_no_loops(test_example, p_class, p_att_given_class)
print('z', z)

[0.23361808 0.19261693 0.57376499]
z 2


## Exercise 1

Write a program to classify the MNIST dataset using the Naive Bayes classifer. 

Read the dataset

In [22]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.float32(x_train/255).reshape(x_train.shape[0],-1)
x_test = np.float32(x_test/255).reshape(x_test.shape[0],-1)

Create a new dataset using binary attributes as follows:

m = mean pixel value in x_train

x_train_binary[i,j]  =  1 if x_train[i,j] >m; otherwise x_train_binary[i,j] = 0

x_test_binary[i,j]  =  1 if x_test[i,j] >m; otherwise x_test_binary[i,j] = 0



# **Note of assignment:**
Note that in this assignment, I calculated the mean pixel of each attribute, instead of getting the overall mean value pixel of the whole training data.
In the end I have 784 mean pixel values, which I will compare with the other values in each attributes of the examples provided in the training data.

In [23]:
# Note that the data is already in a 1-d array of size 784
# The values from the y-value data goes from 0-9


# Creating new dataset
# Gets mean 784 mean values
x_train_mean_pixel = x_train.sum(axis=0)/x_train.shape[0]
x_test_mean_pixel = x_test.sum(axis=0)/x_test.shape[0]

x_train_binary = np.zeros(shape=(x_train.shape[0], x_train.shape[1]))
x_test_binary = np.zeros(shape=(x_test.shape[0], x_test.shape[1]))

for i in range(x_train.shape[1]): # Number of attributes
  x_train_binary[:,i] = np.where(x_train[:,i] <= x_train_mean_pixel[i], 0, 1)

for j in range(x_test.shape[1]): # Number of attributes
  x_test_binary[:,j] = np.where(x_test[:,j] <= x_test_mean_pixel[j], 0, 1)

Compute class probabilities (p_class) from training data, where p_class[i] represents the probability that an example belongs to class i. You can estimate this by dividing the number of instances in your training dataset that belong to class i over the number of training examples.

In [24]:
# Your code here
# Note that there are 10 classes (snekaers, dresses, etc)
class_count_train = np.bincount(y_train)
class_count_test = np.bincount(y_test)
p_class_train = class_count_train / x_train_binary.shape[0]
p_class_test = class_count_test / x_test_binary.shape[0]

print(class_count_train)
print(class_count_test)
print(p_class_train)
print(p_class_test)

[5923 6742 5958 6131 5842 5421 5918 6265 5851 5949]
[ 980 1135 1032 1010  982  892  958 1028  974 1009]
[0.09871667 0.11236667 0.0993     0.10218333 0.09736667 0.09035
 0.09863333 0.10441667 0.09751667 0.09915   ]
[0.098  0.1135 0.1032 0.101  0.0982 0.0892 0.0958 0.1028 0.0974 0.1009]


Compute conditional probabilities (p_att_given_class) from training data, where p_att_given_class[i,j] represents the probability that attribute j in an example of class i is equal to 1. You can estimate this by dividing the number of times attribute j is equal to 1 in training instances of class j over the number of training instances of class j

In [25]:
# Your code here
print(x_train_binary.shape)
# Creates shape 10 classes rows and 784 columns of attributes
p_att_given_class = np.zeros(shape=(p_class_train.shape[0], x_train_binary.shape[1]))

for example in range(x_train_binary.shape[0]):
  class_val = y_train[example]
  for att in range(x_train_binary.shape[1]):
    if x_train_binary[example][att] == 1:
      p_att_given_class[class_val][att] += 1

print('pagc', p_att_given_class)
print('hello', p_att_given_class[:1])
p_att_given_class = p_att_given_class / class_count_train[:, np.newaxis]
print(p_att_given_class)
print(np.argmax(p_att_given_class))

sum = 0
for i in range(p_att_given_class.shape[0]):
  for j in range (p_att_given_class.shape[1]):
    sum += p_att_given_class[i][j]

print(sum)

print('hwllo2', p_att_given_class[:1])

(60000, 784)
pagc [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
hello [[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 1.000e+00
  0.000e+00 1.000e+00 1.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00 1.000e+00
  2.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00 2.000e+00 4.000e+00
  2.000e+00 2.000e+00 4.000e+00 2.000e+00 1.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e

Classify test data - print accuracy and confusion matrix (feel free to use sklearn to print the accuracy and confusion matrix)




In [26]:
# Your code here
# Importing confusion matrix formula
from sklearn.metrics import confusion_matrix

def accuracy_multiclass(preds, Y_test):
  return np.sum(preds == Y_test) / Y_test.shape[0]

preds = np.zeros(y_test.shape[0])
print('binary shape', x_test_binary.shape)
# Going through testing data x_test
for i in range(y_test.shape[0]):
  class_pred = classify(x_test_binary[i], p_class_train, p_att_given_class)
  preds[i] = class_pred

accuracy = accuracy_multiclass(preds, y_test)

print('preds', preds)
print('y_test', y_test)
print('Accuracy: ', accuracy)
print(confusion_matrix(preds, y_test))


binary shape (10000, 784)
preds [7. 2. 1. ... 9. 8. 6.]
y_test [7 2 1 ... 4 5 6]
Accuracy:  0.8373
[[ 875    0   19    5    2   18   18    1   13   10]
 [   0 1082   11   19    8    9   18   29   31   15]
 [   3    7  846   36    8    8   19   13   11    5]
 [   4    4   33  846    1  121    1    6   71    9]
 [   1    0   17    1  769   23    9   16   16   71]
 [  43    4    4   15    2  635   31    2   23    9]
 [  28    5   30    8   16   19  853    0    7    0]
 [   1    0   16   16    1    8    0  862    5   27]
 [  25   33   54   41   20   27    9   28  757   15]
 [   0    0    2   23  155   24    0   71   40  848]]


# **Note on assignment:**
As previously noted, in this assignment I calculated the mean pixel value for each attribute, so in total 784 mean pixel values. This way I achieved better accuracy than if I would have had only one mean pixel value and compare that to every other pixel values.
Also, just for a fun fact I tried 0.5 instead of the mean pixel value, and that actually got a better accuracy 0.84 than trying the previous mean pixel values.

## Submission Instructions

1. File > Download .ipynb
2. Go to Blackboard, find the submission page, and upload the .ipynb file you just downloaded.